In [1]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from database.async_db import asyncHandler as db
from navec import Navec

# 1. Sentence embedding

In [2]:
async def get_vector_data(id_t, id_f):
    t = [await db.get_vector_by_p_id(i) for i in id_t]
    f = [await db.get_vector_by_p_id(i) for i in id_f]
    return t + f, [1] * len(t) + [0] * len(f)

In [3]:
id_t = [255, 67, 35, 446, 382, 498]

id_f = [507, 8005, 2774, 1998, 10806, 11649, 2873, 10135, 9498, 4669, 11457, 11339, 9106, 8793, 2169, 8905, 3790,
        3832, 5273, 10106, 5402, 260, 70, 6270, 3893, 387]

x, y = await get_vector_data(id_t, id_f)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

cat = CatBoostClassifier()

cat.fit(
   X_train, y_train,
   eval_set=(X_test, y_test),
   logging_level='Silent',
   plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

# 2. Word embedding

In [4]:
async def get_word_data(id_t, id_f):
    path = 'hudlit_v1_12B_500K_300d_100q.tar'
    navec = Navec.load(path)

    t = [await navec.get(db.get_lemma_by_p_id(i)) for i in id_t]
    f = [await navec.get(db.get_lemma_by_p_id(i)) for i in id_f]
    return t + f, [1] * len(t) + [0] * len(f)

In [5]:
id_t = [255, 67, 35, 446, 382, 498]

id_f = [507, 8005, 2774, 1998, 10806, 11649, 2873, 10135, 9498, 4669, 11457, 11339, 9106, 8793, 2169, 8905, 3790,
        3832, 5273, 10106, 5402, 260, 70, 6270, 3893, 387]

x, y = await get_word_data(id_t, id_f)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

cat = CatBoostClassifier()

cat.fit(
   X_train, y_train,
   eval_set=(X_test, y_test),
   logging_level='Silent',
   plot=True
)

FileNotFoundError: [Errno 2] No such file or directory: 'hudlit_v1_12B_500K_300d_100q.tar'